In [8]:
import requests
import pandas as pd
from requests.exceptions import RequestException
from typing import List

def get_team_abbrs_list() -> list:
    url = 'https://api.nhle.com/stats/rest/en/team'
    response = requests.get(url)
    response.raise_for_status()
    json_data = response.json()
    return sorted([team['triCode'] for team in json_data['data']])


In [9]:
def get_season_id_list() -> list:
    url = 'https://api-web.nhle.com/v1/season'
    response = requests.get(url)
    response.raise_for_status()
    json_data = response.json()
    return [season for season in json_data if season >= 20202011]


In [10]:
def get_one_season_info(season_id) -> pd.DataFrame:
    url = f'https://api.nhle.com/stats/rest/en/team/summary?sort=shotsForPerGame&cayenneExp=seasonId={season_id}%20and%20gameTypeId=2'
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json().get('data', [])
        return pd.DataFrame(data)
    except RequestException as e:
        print(f'Ошибка при выполнении запроса: {e}')
        return pd.DataFrame()


def get_many_season_info() -> pd.DataFrame:
    season_ids = get_season_id_list()
    df_list = [get_one_season_info(season_id) for season_id in season_ids]
    return pd.concat(df_list, ignore_index=True)


In [11]:
def get_teams() -> pd.DataFrame:
    url = 'https://api.nhle.com/stats/rest/en/team'
    try:
        response = requests.get(url)
        response.raise_for_status()
        json_data = response.json()
    except requests.RequestException as e:
        print(f'Error fetching data: {e}')
        return pd.DataFrame()

    data = pd.json_normalize(json_data['data'])
    columns = ['id', 'franchiseId', 'fullName', 'leagueId', 'rawTricode', 'triCode']
    return data[columns].sort_values(by='id').reset_index(drop=True)


In [12]:
def get_player_stats():
    url = f'https://api-web.nhle.com/v1/club-stats/TOR/20232024/2'
    response = requests.get(url)
    response.raise_for_status()
    json_data = response.json()
    data = pd.json_normalize(json_data['skaters'])
    return data

In [ ]:
def fetch_team_roster(team: str) -> List[dict]:
    url = f'https://api-web.nhle.com/v1/roster/{team}/20232024'
    try:
        response = requests.get(url)
        response.raise_for_status()
        json_data = response.json()
        players = sum((json_data[key] for key in ['forwards', 'defensemen', 'goalies']), [])
        for player in players:
            player['team'] = team
        return players
    except RequestException:
        return []

def get_roster(teams: List[str]) -> pd.DataFrame:
    all_players = [player for team in teams for player in fetch_team_roster(team)]

    if not all_players:
        return pd.DataFrame()  # Возвращаем пустой DataFrame, если нет данных

    data = pd.json_normalize(all_players)
    columns = [
        'id', 'headshot', 'sweaterNumber', 'positionCode',
        'shootsCatches', 'heightInCentimeters', 'weightInKilograms',
        'birthDate', 'birthCountry', 'firstName.default', 'lastName.default',
        'birthCity.default', 'birthStateProvince.default', 'team'
    ]

    return data[columns].sort_values(by='id').reset_index(drop=True)


ts = get_roster(get_team_abbrs_list())
ts